In [1]:
%pip install langfuse
%pip install langchain langgraph langchain_openai langchain_community
%pip install --upgrade --quiet  yandexcloud yandex-cloud-ml-sdk


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


Заполняем словарь документов

In [2]:
documents = [
    {
        "id": 166213,
        "title": "Изменить формат общего шаблона excel при скачивании файлов",
        "url": "https://inside.notamedia.ru/company/personal/user/4658/tasks/task/view/166213/"
    },

    {
        "id": 151543,
        "title": "Реализовать страницу со списком акселераторов",
        "url": "https://inside.notamedia.ru/company/personal/user/4658/tasks/task/view/151543/"
    },

    {
        "id": 196677,
        "title": "[Bug][Major] Доработать статусы заявок в Акселераторах",
        "url": "https://inside.notamedia.ru/company/personal/user/4658/tasks/task/view/196677/"
    },

    {
        "id": 181744,
        "title": "Реализация группы прав \"Менеджер Акселератора партнер\"",
        "url": "https://inside.notamedia.ru/company/personal/user/4658/tasks/task/view/181744/"
    },

    {
        "id": 190897,
        "title": "Доработка кабинета Акселератора, добавление внешних модераторов",
        "url": "https://inside.notamedia.ru/company/personal/user/4658/tasks/task/view/190897/"
    },

    {
        "id": 184464,
        "title": "Доработка ролевой модели в Акселерации (Демо)",
        "url": "https://inside.notamedia.ru/company/personal/user/4658/tasks/task/view/184464/"
    },

    {
        "id": 187834,
        "title": "Добавление рассылки видеозаписи прошедшего мероприятия",
        "url": "https://inside.notamedia.ru/company/personal/user/4658/tasks/task/view/187834/"
    },
]

обогащаем documents текстами задач

In [3]:
import os

# Добавляем поле context с текстом из .md файлов
for doc in documents:
    file_name = f"{doc['id']}.md"
    file_path = os.path.join("tasks", file_name)

    if os.path.exists(file_path):
        with open(file_path, "r", encoding="utf-8") as f:
            doc["context"] = f.read()
    else:
        doc["context"] = ""

    # Создаем поле search как объединение title и context
    doc["search"] = f"{doc['title']} {doc['context']}"



In [4]:
import pandas as pd  # если нет импорта пандаса, то остальной код не имеет смысла

# создаем из наших документов датафрейм
df = pd.DataFrame(documents)
df.head()

,id,title,url,context,search
0,166213,Изменить формат общего шаблона excel при скачи...,https://inside.notamedia.ru/company/personal/u...,Требуется изменить формат excel общего шаблона...,Изменить формат общего шаблона excel при скачи...
1,151543,Реализовать страницу со списком акселераторов,https://inside.notamedia.ru/company/personal/u...,"1. Необходимо сделать активной кнопку ""Акселер...",Реализовать страницу со списком акселераторов ...
2,196677,[Bug][Major] Доработать статусы заявок в Аксел...,https://inside.notamedia.ru/company/personal/u...,Стенд: все площадки\n\n1) Кейс 1\n- Создать Ак...,[Bug][Major] Доработать статусы заявок в Аксел...
3,181744,"Реализация группы прав ""Менеджер Акселератора ...",https://inside.notamedia.ru/company/personal/u...,"Требуется реализовать новую группу прав ""Менед...","Реализация группы прав ""Менеджер Акселератора ..."
4,190897,"Доработка кабинета Акселератора, добавление вн...",https://inside.notamedia.ru/company/personal/u...,Требуется в кабинете Акселератора доработать д...,"Доработка кабинета Акселератора, добавление вн..."


In [5]:
from langchain.document_loaders import DataFrameLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS

loader = DataFrameLoader(df, page_content_column='search')
documents = loader.load()

print(documents)

[Document(metadata={'id': 166213, 'title': 'Изменить формат общего шаблона excel при скачивании файлов', 'url': 'https://inside.notamedia.ru/company/personal/user/4658/tasks/task/view/166213/', 'context': 'Требуется изменить формат excel общего шаблона при скачивании файлов. Изменить с формата "Веб-страница" https://pastenow.ru/GMPC8 на xls.\n\nОтчеты в xls в блоке Бизнес-образования https://pastenow.ru/GMPBC\n\nОценка: 1 р.д.'}, page_content='Изменить формат общего шаблона excel при скачивании файлов Требуется изменить формат excel общего шаблона при скачивании файлов. Изменить с формата "Веб-страница" https://pastenow.ru/GMPC8 на xls.\n\nОтчеты в xls в блоке Бизнес-образования https://pastenow.ru/GMPBC\n\nОценка: 1 р.д.'), Document(metadata={'id': 151543, 'title': 'Реализовать страницу со списком акселераторов', 'url': 'https://inside.notamedia.ru/company/personal/user/4658/tasks/task/view/151543/', 'context': '1. Необходимо сделать активной кнопку "Акселерация" в разделе Бизнес-комп

In [6]:
# # создаем сплиттер документов, чтобы уложиться в лимит по токенам, в нашем случае это не очень полезный шаг

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

print(texts)

[Document(metadata={'id': 166213, 'title': 'Изменить формат общего шаблона excel при скачивании файлов', 'url': 'https://inside.notamedia.ru/company/personal/user/4658/tasks/task/view/166213/', 'context': 'Требуется изменить формат excel общего шаблона при скачивании файлов. Изменить с формата "Веб-страница" https://pastenow.ru/GMPC8 на xls.\n\nОтчеты в xls в блоке Бизнес-образования https://pastenow.ru/GMPBC\n\nОценка: 1 р.д.'}, page_content='Изменить формат общего шаблона excel при скачивании файлов Требуется изменить формат excel общего шаблона при скачивании файлов. Изменить с формата "Веб-страница" https://pastenow.ru/GMPC8 на xls.\n\nОтчеты в xls в блоке Бизнес-образования https://pastenow.ru/GMPBC\n\nОценка: 1 р.д.'), Document(metadata={'id': 151543, 'title': 'Реализовать страницу со списком акселераторов', 'url': 'https://inside.notamedia.ru/company/personal/user/4658/tasks/task/view/151543/', 'context': '1. Необходимо сделать активной кнопку "Акселерация" в разделе Бизнес-комп

In [7]:
import os

YC_FOLDER_ID = os.getenv("folder_id")
YC_API_KEY = os.getenv("ya_token")


In [8]:
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings.yandex import YandexGPTEmbeddings

# Создаем модель для генерации эмбеддингов
yandex_embeddings = YandexGPTEmbeddings(model="doc", folder_id=YC_FOLDER_ID, iam_token=YC_API_KEY)


Создаем индекс FAISS

In [31]:
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings.yandex import YandexGPTEmbeddings

# Создаем модель для генерации эмбеддингов
yandex_embeddings = YandexGPTEmbeddings(model="doc", folder_id=YC_FOLDER_ID, iam_token=YC_API_KEY)

# Проверяем наличие локального индекса
index_path = "faiss_index"
if os.path.exists(index_path):
    print(f"Загрузка существующего индекса из {index_path}")
    faiss_index = FAISS.load_local(index_path, yandex_embeddings, allow_dangerous_deserialization=True)
else:
    print("Создание нового индекса")
    # Создание векторного хранилища FAISS
    faiss_index = FAISS.from_texts(
        texts=[doc.page_content for doc in texts],  # Тексты для индексирования
        embedding=yandex_embeddings,  # Экземпляр Embeddings
        metadatas=[{"id": doc.metadata.get("id", ""), "url": doc.metadata.get("url", ""),
                    "title": doc.metadata.get("title", "")} for doc in texts]  # Метаданные
    )
    # Сохранение индекса локально
    faiss_index.save_local(index_path)



Загрузка существующего индекса из faiss_index


Проверяем индекс

In [42]:
query = 'Какие поля есть в карточке мероприятия?'

In [43]:
relevants_score = faiss_index.similarity_search_with_score(query)
print(relevants_score)

[(Document(id='ed49e0d4-8a2f-45ff-adf3-2318fbe8f385', metadata={'id': 187834, 'url': 'https://inside.notamedia.ru/company/personal/user/4658/tasks/task/view/187834/', 'title': 'Добавление рассылки видеозаписи прошедшего мероприятия'}, page_content='Добавление рассылки видеозаписи прошедшего мероприятия Требуется добавить элемент по управлению рассылками видеозаписи прошедшего мероприятия\n\n1. В карточку мероприятия добавить возможность прикрепление обработанной видеозаписи (смонтированной) и записи после мероприятия (автоматическая запись трансляции)\nДоработать карточку мероприятия, добавить:\n1. 1. Чек-бокс "Добавить видеозапись после завершения мероприятия (автоматическая запись трансляции)";\n1. 2. Поле "Добавить видеозапись после завершения мероприятия (Обработанная видеозапись)"\nСтандартное поле для загрузки видео (как в акселераторе и видеолекциях), добавление через загрузку с ПК или медиабиблиотеку\nСнимок экрана 2023-02-11 в 16.57.29.png\n\n\n1. 3. Новые поля разместить посл

Создаем модель

In [44]:
from langchain_community.llms import YandexGPT

llm = YandexGPT(
    folder_id=YC_FOLDER_ID,
    iam_token=YC_API_KEY,
    model="yandexgpt",
    temperature=0.3,
    max_tokens=500
)

Проверяем на запросе с использованием FAISS

In [45]:
from langchain.chains import RetrievalQA

# создаем цепочку
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type='stuff',
    retriever=faiss_index.as_retriever()
)

qa_chain.invoke(query)

{'query': 'Какие поля есть в карточке мероприятия?',
 'result': 'В карточке мероприятия есть следующие поля:\n* Краткое описание — анонс;\n* Дата старта сбора заявок;\n* Дата окончания сбора заявок;\n* Продление сбора заявок (чекбокс);\n* Количество зарегистрированных пользователей;\n* Чек-бокс «Добавить видеозапись после завершения мероприятия (автоматическая запись трансляции)»;\n* Поле «Добавить видеозапись после завершения мероприятия (Обработанная видеозапись)»;\n* Кнопка «Перейти в мероприятия».\n\nТакже планируется добавить вывод загруженного видео и уведомление о записи прошедшего мероприятия.'}

Подключаем промпт

In [50]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

# Создаем шаблон промпта с полями из метаданных
prompt_template = PromptTemplate(
    input_variables=["question", "context"],
    template="""Ответь на вопрос, используя следующие документы:

Контекст:
{context}

Вопрос:
{question}

Ответ:"""
)

# цепочка с кастомным промтом
llm_chain = LLMChain(llm=llm, prompt=prompt_template)

# Получаем релевантные документы из FAISS
relevants = faiss_index.similarity_search(query, k=2)

# Формируем контекст и метаданные
context = "\n\n".join([doc.page_content for doc in relevants])

# Отправляем запрос в LLMChain
result = llm_chain.invoke({
    "question": query,
    "context": context
})

print("Вопрос:")
print(query)

print("Ответ:")
print(result["text"])

print(f"\n\nПодробнее в задачах:\n")
for doc in relevants:
    print(f"{doc.metadata["title"]}\n{doc.metadata["url"]}\n")


Вопрос:
Какие поля есть в карточке мероприятия?
Ответ:
В карточке мероприятия есть следующие поля:
* Краткое описание — анонс.
* Дата старта сбора заявок.
* Дата окончания сбора заявок.
* Чек-бокс «Продление сбора заявок».
* Поле для указания даты окончания сбора заявок в случае продления.
* Поле для загрузки обработанной видеозаписи (смонтированной).
* Чек-бокс «Добавить видеозапись после завершения мероприятия (автоматическая запись трансляции)».
* Поле для загрузки видеозаписи после мероприятия (автоматическая запись трансляции).
* Поле «Количество зарегистрированных пользователей».


Подробнее в задачах:

Добавление рассылки видеозаписи прошедшего мероприятия
https://inside.notamedia.ru/company/personal/user/4658/tasks/task/view/187834/

Реализовать страницу со списком акселераторов
https://inside.notamedia.ru/company/personal/user/4658/tasks/task/view/151543/



Добавляем мониторинг с помощью Langfuse

In [39]:
import os

LF_PUBLIC_KEY = os.getenv("LANGFUSE_PUBLIC_KEY")
LF_SECRET_KEY = os.getenv("LANGFUSE_SECRET_KEY")


In [40]:
from langchain.chains import LLMChain
from langfuse import Langfuse

# Инициализация Langfuse
langfuse = Langfuse(
    public_key=LF_PUBLIC_KEY,
    secret_key=LF_SECRET_KEY,
    host="https://us.cloud.langfuse.com"
)


In [51]:
from langfuse import get_client, Langfuse
from langfuse.langchain import CallbackHandler
import time

langfuse = get_client()

# Generate deterministic trace ID from external system
external_request_id = "home_request_15"
predefined_trace_id = Langfuse.create_trace_id(seed=external_request_id)

langfuse_handler = CallbackHandler()

# Отслеживание времени выполнения
start_time = time.time()  # Начало всей операции


# Разделяем документы и оценки
documents = [doc for doc, score in relevants_score]
scores = [score for doc, score in relevants_score]

# Отслеживание количества и релевантности
document_count = len(documents)
average_relevance = sum(scores) / len(scores) if document_count > 0 else 0

with langfuse.start_as_current_span(
        name="langchain-request-1",
        trace_context={"trace_id": predefined_trace_id}
) as span:
    span.update_trace(
        user_id="user_123",
        input={
            "document_ids": [doc.metadata["id"] for doc in relevants],
            "document_count": document_count,
            "relevance_scores": scores,
            "average_relevance": average_relevance

        }
    )

    # Замер времени начала выполнения LLM-цепочки
    llm_start_time = time.time()

    response = llm_chain.invoke(
        {
            "question": query,
            "context": context
        },
        config={
            "callbacks": [langfuse_handler],
            "metadata": {
                "langfuse_user_id": "random-user",
                "langfuse_session_id": "random-session",
                "langfuse_tags": ["random-tag-1", "random-tag-2"],
                "model_name": "yandexgpt"
            }
        }
    )



    # Замер времени окончания выполнения LLM-цепочки
    llm_end_time = time.time()
    llm_duration = llm_end_time - llm_start_time  # Время выполнения LLM

    # Общее время выполнения
    total_end_time = time.time()
    total_duration = total_end_time - start_time  # Общее время операции

    span.update_trace(
        output={
            "response": response,
            "llm_duration_seconds": llm_duration,  # Время работы LLM
            "total_duration_seconds": total_duration,
            "document_count": document_count,
            "relevance_scores": scores,
            "average_relevance": average_relevance  # Общее время
        }
    )

print("Вопрос:")
print(query)

print("Ответ:")
print(response["text"])

print(f"\n\nПодробнее в задачах:\n")
for doc in relevants:
    print(f"{doc.metadata["title"]}\n{doc.metadata["url"]}\n")

print(f"Trace ID: {predefined_trace_id}")
print(f"Trace ID: {langfuse_handler.last_trace_id}")
print(f"LLM Duration: {llm_duration:.4f} seconds")
print(f"Total Duration: {total_duration:.4f} seconds")
print(f"Document Count: {document_count}")
print(f"Average Relevance Score: {average_relevance:.4f}")
print(f"Relevance Scores: {scores}")



Langfuse was not able to parse the LLM model. The LLM call will be recorded without model name. Please create an issue: https://github.com/langfuse/langfuse/issues/new/choose


Вопрос:
Какие поля есть в карточке мероприятия?
Ответ:
В карточке мероприятия есть следующие поля:
* Краткое описание — анонс.
* Дата старта сбора заявок.
* Дата окончания сбора заявок.
* Чек-бокс «Продление сбора заявок».
* Поле для указания даты окончания сбора заявок в случае продления.
* Поле для загрузки обработанной видеозаписи (смонтированной).
* Чек-бокс «Добавить видеозапись после завершения мероприятия (автоматическая запись трансляции)».
* Поле для загрузки автоматической видеозаписи после завершения мероприятия (автоматическая запись трансляции).
* Поле «Количество зарегистрированных пользователей».


Подробнее в задачах:

Добавление рассылки видеозаписи прошедшего мероприятия
https://inside.notamedia.ru/company/personal/user/4658/tasks/task/view/187834/

Реализовать страницу со списком акселераторов
https://inside.notamedia.ru/company/personal/user/4658/tasks/task/view/151543/

Trace ID: ceb21301244edc4abae0cce37d7759b5
Trace ID: ceb21301244edc4abae0cce37d7759b5
LLM Durati